# Circuit Training - Optimizing a Quantum Circuit
## [QHack 2021](https://challenge.qhack.ai/)
### Matt Wright

In [1]:
from pennylane import numpy as np
import pennylane as qml

In [2]:
WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

In [3]:
def variational_circuit(params):
    """
    # DO NOT MODIFY anything in this function! It is used to judge your solution.

    This is a template variational quantum circuit containing a fixed layout of gates with variable
    parameters. To be used as a QNode, it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function (as shown above).

    The output of this circuit is the expectation value of a Hamiltonian. An unknown Hamiltonian
    will be used to judge your solution.

    Args:
        params (np.ndarray): An array of optimizable parameters of shape (30,)
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires=[0, 1]))

In [4]:
def parameter_shift(circuit, params):
    """Compute the gradient of the variational circuit given by using the 
    parameter-shift rule. See my solution in `parameter_shift.ipynb`

    Args:
        circuit (QNode): A variational circuit to compute the gradient of.
        weights (array): An array of floating-point numbers with size (2, 3).

    Returns:
        array: The gradient of the variational circuit. The shape should match
        the input weights array.
    """
    
    s = np.pi/2  # shift factor
    gradient = np.zeros_like(params) 
    
    for i in range(params.size):
        unit_shift = np.zeros_like(params)
        unit_shift[i] = s

        forward_shift = circuit(params + unit_shift)
        backward_shift = circuit(params - unit_shift)
        gradient[i] = (forward_shift - backward_shift) / (2 * np.sin(s))
    
    return gradient

In [5]:
def optimize_circuit(params):
    """Minimize the variational circuit parameters using an elementary optimizer I built 
    from stract and return its minimum value.

    Args:
        params (np.ndarray): Input parameters to be optimized, of dimension 30

    Returns:
        float: the value of the optimized QNode
    """

    optimal_value = 0.0

    # My solution begins here #
    # Initialize the device
    dev = qml.device('default.qubit', wires=WIRES)

    # Instantiate the QNode
    circuit = qml.QNode(variational_circuit, dev)

    # Minimize the circuit
    steps = 75
    step_size = 0.75
    
    from time import time
    ts = time()
    for i in range(steps):
        gradient = parameter_shift(circuit, params)
        step = step_size * gradient
        params -= step

    optimal_value = circuit(params)
    
    # My solution ends here #
    return optimal_value

## Testing

In [6]:
from test_result import test_result

In [7]:
tol = 5 * 10e-3

# Generate random initial parameters
np.random.seed(1967)

print('Test 1:')
hamiltonian = [0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]
hamiltonian = np.array(hamiltonian, float).reshape((2 ** WIRES, 2 ** WIRES))

initial_params = np.random.random(NUM_PARAMETERS)

pred = optimize_circuit(initial_params).item()
ans = 0.6174534088307637
test_result(pred, ans, tol)

print('\nTest 2:')
hamiltonian = [0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]
hamiltonian = np.array(hamiltonian, float).reshape((2 ** WIRES, 2 ** WIRES))

initial_params = np.random.random(NUM_PARAMETERS)

pred = optimize_circuit(initial_params).item()
ans = 0.0024648812008861154
test_result(pred, ans, tol)

Test 1:
Success!!

Test 2:
Success!!
